# Transfer Learning MNIST

* Train a simple convnet on the MNIST dataset the first 5 digits [0..4].
* Freeze convolutional layers and fine-tune dense layers for the classification of digits [5..9].

## 1. Import necessary libraries for the model

In [0]:
from keras.datasets import mnist

In [281]:
from google.colab import drive
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Import MNIST data and create 2 datasets with one dataset having digits from 0 to 4 and other from 5 to 9 

In [0]:

(X_train, y_train), (X_test, y_test) = mnist.load_data()

import numpy as np

train_filter = np.where((y_train == 0 ) | (y_train==1)| (y_train ==2)| (y_train==3) | (y_train == 4))
test_filter = np.where((y_test == 0) | (y_test ==1)|(y_test==2)|(y_test==3)|(y_test == 4))

X_train_1, y_train_1 = X_train[train_filter], y_train[train_filter]
X_test_1, y_test_1 = X_test[test_filter], y_test[test_filter]



In [0]:
#using another method to separate the data

train_mask = np.isin(y_train, [5, 9])
test_mask = np.isin(y_test, [5, 9])

X_train_2, y_train_2 = X_train[train_mask], y_train[train_mask]
X_test_2, y_test_2 = X_test[test_mask], y_test[test_mask]


In [284]:

print(X_train_1.shape, X_test_1.shape)
print(X_train_2.shape, X_test_2.shape)
print(Y_train_1.shape, Y_test_1.shape)
print(Y_train_2.shape, Y_test_2.shape)

((30596, 28, 28), (5139, 28, 28))
((11370, 28, 28), (1901, 28, 28))
((30596,), (5139,))
((11370,), (1901,))


## 3. Print x_train, y_train, x_test and y_test for both the datasets

In [285]:
X_train_1
X_train_1.shape

(30596, 28, 28)

In [286]:
X_train_2.shape

(11370, 28, 28)

In [287]:
print(y_train_1, y_train_2)

(array([0, 4, 1, ..., 2, 1, 3], dtype=uint8), array([5, 9, 5, ..., 9, 5, 5], dtype=uint8))


In [288]:
X_train[0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
         18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,
          0,   0],
       [  

In [289]:
X_train_1[0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,  51, 159, 253, 159,  50,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,  48, 238, 252, 252, 252, 237,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  

In [290]:
y_train[0]

5

## ** 4. Let us take only the dataset (x_train, y_train, x_test, y_test) for Integers 0 to 4 in MNIST **
## Reshape x_train and x_test to a 4 Dimensional array (channel = 1) to pass it into a Conv2D layer

In [0]:
X_train_1 = X_train_1.reshape(X_train_1.shape[0],28,28,1).astype('float32')
X_test_1 = X_test_1.reshape(X_test_1.shape[0],28,28,1).astype('float32')

In [0]:
X_train_2 = X_train_2.reshape(X_train_2.shape[0],28,28,1).astype('float32')
X_test_2 = X_test_2.reshape(X_test_2.shape[0],28,28,1).astype('float32')

## 5. Normalize x_train and x_test by dividing it by 255

In [0]:
X_train_1 /= 255
X_test_1 /= 255


## 6. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
import keras
y_train_1 = keras.utils.to_categorical(y_train_1,num_classes=5) 
y_test_1 = keras.utils.to_categorical(y_test_1,num_classes=5) 


In [0]:
y_train_remapped = y_train_2 - 5

In [0]:
y_test_remapped = y_test_2 - 5

In [0]:

y_train_2 = keras.utils.to_categorical(y_train_remapped,num_classes=5)
y_test_2 = keras.utils.to_categorical(y_test_remapped,num_classes=5)

## 7. Build a sequential model with 2 Convolutional layers with 32 kernels of size (3,3) followed by a Max pooling layer of size (2,2) followed by a drop out layer to be trained for classification of digits 0-4  

In [0]:
TRAIN = False
BATCH_SIZE = 28
EPOCHS = 10

In [299]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, Convolution2D


# Define model
model = Sequential()

# 1st Conv Layer
model.add(Convolution2D(32, 3, 3, input_shape=(28, 28, 1), name='conv_1'))
model.add(Activation('relu'))
    

#max pool layer
model.add(MaxPooling2D(pool_size=(2,2), name='Max_pool'))
    
#dropout layer
model.add(Dropout(0.2, name='Dropout_1'))
    

   
    

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), name="conv_1", input_shape=(28, 28, 1...)`
  # Remove the CWD from sys.path while we load stuff.


## 8. Post that flatten the data and add 2 Dense layers with 128 neurons and neurons = output classes with activation = 'relu' and 'softmax' respectively. Add dropout layer inbetween if necessary  

In [300]:
# Fully Connected Layer
model.add(Flatten())
model.add(Dense(128,name='dense_full_layer'))
model.add(Activation('relu'))

# Prediction Layer
model.add(Dense(5, name='dense_prediction'))
model.add(Activation('softmax'))

# Loss and Optimizer #put metrics as f1 score
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
# Store Training Results
early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
callback_list = [early_stopping]

# Train the model
model.fit(X_train_1, y_train_1, batch_size=BATCH_SIZE, nb_epoch=EPOCHS, 
              validation_data=(X_test_1, y_test_1), callbacks=callback_list) 

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:18: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 30596 samples, validate on 5139 samples
Epoch 1/10
30596/30596 [==============================] - 9s 288us/step - loss: 0.0764 - acc: 0.9771 - val_loss: 0.0163 - val_acc: 0.9947
Epoch 2/10
30596/30596 [==============================] - 8s 259us/step - loss: 0.0237 - acc: 0.9926 - val_loss: 0.0106 - val_acc: 0.9963
Epoch 3/10
30596/30596 [==============================] - 8s 260us/step - loss: 0.0138 - acc: 0.9961 - val_loss: 0.0149 - val_acc: 0.9947
Epoch 4/10
30596/30596 [==============================] - 8s 259us/step - loss: 0.0083 - acc: 0.9973 - val_loss: 0.0121 - val_acc: 0.9959
Epoch 5/10
30596/30596 [==============================] - 8s 259us/step - loss: 0.0073 - acc: 0.9972 - val_loss: 0.0082 - val_acc: 0.9971
Epoch 6/10
30596/30596 [==============================] - 8s 259us/step - loss: 0.0043 - acc: 0.9984 - val_loss: 0.0083 - val_acc: 0.9979
Epoch 7/10
30596/30596 [==============================] - 8s 258us/step - loss: 0.0034 - acc: 0.9989 - val_loss: 0.0089 - v

## 9. Print the training and test accuracy

In [301]:
score = model.evaluate(X_test_1, y_test_1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

5139/5139 [==============================] - 0s 76us/step
('Test loss:', 0.007064032351169863)
('Test accuracy:', 0.9976649153531816)


## 10. Make only the dense layers to be trainable and convolutional layers to be non-trainable

In [302]:
#Freezing layers in the model which don't have 'dense' in their name
for layer in model.layers:
  if('dense' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in model.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

conv_1
False
activation_51
False
Max_pool
False
Dropout_1
False
flatten_22
False
dense_full_layer
True
activation_52
False
dense_prediction
True
activation_53
False


## 11. Use the model trained on 0 to 4 digit classification and train it on the dataset which has digits 5 to 9  (Using Transfer learning keeping only the dense layers to be trainable)

In [0]:
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

#To use adam optimizer for learning weights with learning rate = 0.001
optimizer = Adam(lr=0.0001)
#Set the loss function and optimizer for the model training
model.compile(loss=categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy'])

In [304]:
#Training on the dataset
model.fit(X_train_2, y_train_2,
          batch_size=32,
          epochs=20,
          verbose=1,
          validation_data=(X_test_2, y_test_2))

Train on 11370 samples, validate on 1901 samples
Epoch 1/20
11370/11370 [==============================] - 3s 269us/step - loss: 7.2904 - acc: 0.5472 - val_loss: 6.8366 - val_acc: 0.5750
Epoch 2/20
11370/11370 [==============================] - 2s 190us/step - loss: 6.0315 - acc: 0.6254 - val_loss: 5.1563 - val_acc: 0.6791
Epoch 3/20
11370/11370 [==============================] - 2s 189us/step - loss: 4.2010 - acc: 0.7388 - val_loss: 3.6071 - val_acc: 0.7759
Epoch 4/20
11370/11370 [==============================] - 2s 191us/step - loss: 2.5595 - acc: 0.8404 - val_loss: 1.8738 - val_acc: 0.8837
Epoch 5/20
11370/11370 [==============================] - 2s 190us/step - loss: 1.8350 - acc: 0.8858 - val_loss: 1.3396 - val_acc: 0.9169
Epoch 6/20
11370/11370 [==============================] - 2s 192us/step - loss: 1.2966 - acc: 0.9194 - val_loss: 0.9553 - val_acc: 0.9406
Epoch 7/20
11370/11370 [==============================] - 2s 190us/step - loss: 1.1239 - acc: 0.9300 - val_loss: 0.9425 - v

## 12. Print the accuracy for classification of digits 5 to 9

In [305]:
score = model.evaluate(X_test_2, y_test_2)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

1901/1901 [==============================] - 0s 80us/step
('Test loss:', 0.48328861476621954)
('Test accuracy:', 0.9700157811678064)


## Sentiment analysis <br> 

The objective of the second problem is to perform Sentiment analysis from the tweets data collected from the users targeted at various mobile devices.
Based on the tweet posted by a user (text), we will classify if the sentiment of the user targeted at a particular mobile device is positive or not.

### 13. Read the dataset (tweets.csv) and drop the NA's while reading the dataset

In [0]:
tweet_df = pd.read_csv('/content/drive/My Drive/deep learning/Residency 8/Internal Lab/tweets.csv')

### 14. Preprocess the text and add the preprocessed text in a column with name `text` in the dataframe.

In [0]:
def preprocess(text):
    try:
        return text.decode('ascii')
    except Exception as e:
        return ""

In [0]:
tweet_df['text'] = [preprocess(text) for text in tweet_df.tweet_text]

In [354]:
tweet_df.head(10)

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,text
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,.@wesley83 I have a 3G iPhone. After 3 hrs twe...
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,@jessedee Know about @fludapp ? Awesome iPad/i...
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,@swonderlin Can not wait for #iPad 2 also. The...
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,@sxsw I hope this year's festival isn't as cra...
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,@sxtxstate great stuff on Fri #SXSW: Marissa M...
5,@teachntech00 New iPad Apps For #SpeechTherapy...,NaN,No emotion toward brand or product,@teachntech00 New iPad Apps For #SpeechTherapy...
6,NaN,NaN,No emotion toward brand or product,
7,"#SXSW is just starting, #CTIA is around the co...",Android,Positive emotion,"#SXSW is just starting, #CTIA is around the co..."
8,Beautifully smart and simple idea RT @madebyma...,iPad or iPhone App,Positive emotion,Beautifully smart and simple idea RT @madebyma...
9,Counting down the days to #sxsw plus strong Ca...,Apple,Positive emotion,Counting down the days to #sxsw plus strong Ca...


### 15. Consider only rows having Positive emotion and Negative emotion and remove other rows from the dataframe.

In [0]:
filter_tweet_df = tweet_df.loc[(tweet_df['is_there_an_emotion_directed_at_a_brand_or_product'] == "Positive emotion")| (tweet_df['is_there_an_emotion_directed_at_a_brand_or_product'] == "Negative emotion")] 

In [356]:
filter_tweet_df.head(10)

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,text
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,.@wesley83 I have a 3G iPhone. After 3 hrs twe...
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,@jessedee Know about @fludapp ? Awesome iPad/i...
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,@swonderlin Can not wait for #iPad 2 also. The...
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,@sxsw I hope this year's festival isn't as cra...
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,@sxtxstate great stuff on Fri #SXSW: Marissa M...
7,"#SXSW is just starting, #CTIA is around the co...",Android,Positive emotion,"#SXSW is just starting, #CTIA is around the co..."
8,Beautifully smart and simple idea RT @madebyma...,iPad or iPhone App,Positive emotion,Beautifully smart and simple idea RT @madebyma...
9,Counting down the days to #sxsw plus strong Ca...,Apple,Positive emotion,Counting down the days to #sxsw plus strong Ca...
10,Excited to meet the @samsungmobileus at #sxsw ...,Android,Positive emotion,Excited to meet the @samsungmobileus at #sxsw ...
11,Find &amp; Start Impromptu Parties at #SXSW Wi...,Android App,Positive emotion,Find &amp; Start Impromptu Parties at #SXSW Wi...


### 16. Represent text as numerical data using `CountVectorizer` and get the document term frequency matrix

#### Use `vect` as the variable name for initialising CountVectorizer.

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [0]:
tf = pd.DataFrame(vect.fit_transform(filter_tweet_df['text']).toarray(), columns=vect.get_feature_names())


### 17. Find number of different words in vocabulary

In [359]:
tf.head(10)

,000,02,03,0310apple,08,10,100,100s,100tc,101,...,zimride,zing,zip,zite,zms,zombies,zomg,zone,zoom,zzzs
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Tip: To see all available functions for an Object use dir

### 18. Find out how many Positive and Negative emotions are there.

Hint: Use value_counts on that column

In [360]:
filter_tweet_df['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

Positive emotion    2978
Negative emotion     570
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

### 19. Change the labels for Positive and Negative emotions as 1 and 0 respectively and store in a different column in the same dataframe named 'Label'

Hint: use map on that column and give labels

In [0]:
def sentiment(tweet):
  if tweet == "Positive emotion":
    return 1
  else:
    return 0
  
  

In [362]:

filter_tweet_df['is_there_an_emotion_directed_at_a_brand_or_product'] = map(sentiment,filter_tweet_df['is_there_an_emotion_directed_at_a_brand_or_product'])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [363]:
filter_tweet_df.head(10)

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,text
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,1,@jessedee Know about @fludapp ? Awesome iPad/i...
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,1,@swonderlin Can not wait for #iPad 2 also. The...
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,0,@sxsw I hope this year's festival isn't as cra...
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,1,@sxtxstate great stuff on Fri #SXSW: Marissa M...
7,"#SXSW is just starting, #CTIA is around the co...",Android,1,"#SXSW is just starting, #CTIA is around the co..."
8,Beautifully smart and simple idea RT @madebyma...,iPad or iPhone App,1,Beautifully smart and simple idea RT @madebyma...
9,Counting down the days to #sxsw plus strong Ca...,Apple,1,Counting down the days to #sxsw plus strong Ca...
10,Excited to meet the @samsungmobileus at #sxsw ...,Android,1,Excited to meet the @samsungmobileus at #sxsw ...
11,Find &amp; Start Impromptu Parties at #SXSW Wi...,Android App,1,Find &amp; Start Impromptu Parties at #SXSW Wi...


### 20. Define the feature set (independent variable or X) to be `text` column and `labels` as target (or dependent variable)  and divide into train and test datasets

In [0]:
X = filter_tweet_df['text']
y = filter_tweet_df['is_there_an_emotion_directed_at_a_brand_or_product']

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 1)

## 21. **Predicting the sentiment:**


### Use Naive Bayes and Logistic Regression and their accuracy scores for predicting the sentiment of the given text

In [366]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

# create document-term matrices
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

# use Naive Bayes to predict the star rating
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)

# calculate accuracy
print(metrics.accuracy_score(y_test, y_pred_class))

0.8647125140924464


In [367]:
# use logistic regression with text column only
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train_dtm, y_train)
y_pred_class = logreg.predict(X_test_dtm)
print metrics.accuracy_score(y_test, y_pred_class)

0.8523111612175873


## 22. Create a function called `tokenize_predict` which can take count vectorizer object as input and prints the accuracy for x (text) and y (labels)

In [0]:
def tokenize_test(vect):
    x_train_dtm = vect.fit_transform(X_train)
    print('Features: ', x_train_dtm.shape[1])
    x_test_dtm = vect.transform(X_test)
    nb = MultinomialNB()
    nb.fit(x_train_dtm, y_train)
    y_pred_class = nb.predict(x_test_dtm)
    print('Accuracy: ', metrics.accuracy_score(y_test, y_pred_class))

In [369]:
tokenize_test(vect)

('Features: ', 5046)
('Accuracy: ', 0.8647125140924464)


### Create a count vectorizer function which includes n_grams = 1,2  and pass it to tokenize_predict function to print the accuracy score

In [370]:
vect1 = CountVectorizer(ngram_range = (1,2))
tokenize_test(vect1)

('Features: ', 25736)
('Accuracy: ', 0.874859075535513)


### Create a count vectorizer function with stopwords = 'english'  and pass it to tokenize_predict function to print the accuracy score

In [372]:
vect2 = CountVectorizer(stop_words = 'english')
tokenize_test(vect2)

('Features: ', 4806)
('Accuracy: ', 0.8624577226606539)


### Create a count vectorizer function with stopwords = 'english' and max_features =300  and pass it to tokenize_predict function to print the accuracy score

In [373]:
vect3 = CountVectorizer(stop_words = 'english', max_features = 300)
tokenize_test(vect3)

('Features: ', 300)
('Accuracy: ', 0.8094701240135288)


### Create a count vectorizer function with n_grams = 1,2  and max_features = 15000  and pass it to tokenize_predict function to print the accuracy score

In [375]:
vect4 = CountVectorizer(ngram_range = (1,2), max_features = 15000)
tokenize_test(vect4)

('Features: ', 15000)
('Accuracy: ', 0.8771138669673055)


### Create a count vectorizer function with n_grams = 1,2  and include terms that appear at least 2 times (min_df = 2)  and pass it to tokenize_predict function to print the accuracy score

In [376]:
vect5 = CountVectorizer(stop_words = 'english', min_df =2)
tokenize_test(vect5)

('Features: ', 2247)
('Accuracy: ', 0.8500563697857948)
